# Processing quantitative survey data

<br>

**Language: Python**

This notebook shows the process used for cleaning the quantitative data from the Qualtrics survey (ratings and demographic data) for future statistical analysis.


**Notebook contents:**
- [Initial setup](#Initial-setup)
- [Pilot data](#Pilot-data)
- [Survey data](#Survey-data)
- [Final dataframe](#Final-dataframe)
- [Outlier](#Outlier)
- [MFRM preparation](#MFRM-preparation)

## Initial setup

In [1]:
# Import necessary modules

import pandas as pd
import pprint
from IPython.core.interactiveshell import InteractiveShell
import csv
import joblib
import numpy as np

In [2]:
# Set preferred notebook format

%pprint # Turn off pretty printing
InteractiveShell.ast_node_interactivity = "all" # Show all output, not just last item
pd.set_option('display.max_columns', 999) # Allow viewing of all columns

Pretty printing has been turned OFF


## Pilot data

The pilot data consists of 11 survey responses. As there were no major changes to be made based on the pilot feedback, these data are incorporated with the main survey data. In this public version of the data, irrelevant columns and columns including private data (names, emails, etc.) have been removed.

In [3]:
# Read in pilot data (exported from Qualtrics)

pilot = pd.read_csv("../docs/pilot_data.csv")
pilot.head(4)

,Duration (in seconds),Analytic 1_1,Analytic 1_2,Analytic 1_3,Analytic 1_4,Holistic 1_2,Rationale 1,Analytic 2_1,Analytic 2_2,Analytic 2_3,Analytic 2_4,Holistic 2_1,Rationale 2,Analytic 3_1,Analytic 3_2,Analytic 3_3,Analytic 3_4,Holistic 3_1,Rationale 3,Gender,Age,Education,Education_6_TEXT,TESOL cert,TESOL cert_6_TEXT,ELT experience,IELTS status,IELTS status_8_TEXT,IELTS experience,L1,Student L1,FL_148_DO,B1texts_DO,B2texts_DO,C1texts_DO
0,2499,4,4,4,3+,34,The candidate uses only basic vocabulary repet...,5,6,6+,6+,63,The candidate takes risks and attempts to use ...,7,6,8-,7,76,The candidate uses a wide range of vocabulary ...,Female,30-39,Bachelor's degree,NaN,Cambridge Delta,NaN,6-10,"Former IELTS writing examiner,Former IELTS spe...",NaN,6-10,Spanish,Arabic,FL_147|FL_144|FL_146,Text1,Text15,Text23
1,2439,5-,4,4,3+,37,There were a few flashes of less common langua...,7,6+,7+,7,78,"There's a good range, including less common it...",8+,8,8+,8,90,"They're clearly very strong, a wide range of l...",Male,30-39,Master's degree,NaN,"Cambridge CELTA,Cambridge Delta",NaN,11-20,"Former IELTS writing examiner,Former IELTS spe...",NaN,3-5,English,I taught and examined mostly Italian students,FL_144|FL_147|FL_146,Text3,Text16,Text26
2,1111,5,4,5,4,44,Just sufficient for the task. Some error.,5,5+,6,6,56,Some good original lexis but relies heavily on...,8,8,8,8,88,Precise and wide resource. Very good colocatio...,Prefer not to say,60-69,Master's degree,NaN,Trinity CertTESOL,NaN,20+,IELTS Examiner Trainer,NaN,11-20,English,Spanish,FL_146|FL_147|FL_144,Text9,Text12,Text33
3,2068,4-,4,4-,4-,39,basic vocabulary\nsome items not related to ta...,7,7,7-,7-,65,sufficient range to allow some flexibility - d...,8,8,8,8,81,uses a wide range to covey precise meaning - p...,Male,50-59,Master's degree,NaN,"Cambridge CELTA,Cambridge Delta",NaN,20+,"Current IELTS writing examiner,Current IELTS s...",NaN,11-20,English,Korean,FL_144|FL_146|FL_147,Text5,Text13,Text29


In [4]:
# Rename columns

pilot = pilot.rename(columns={'Duration (in seconds)':'duration_sec',
                              'Analytic 1_1':'B1_TR',
                              'Analytic 1_2':'B1_CC',
                              'Analytic 1_3':'B1_LR',
                              'Analytic 1_4':'B1_GRA',
                              'Holistic 1_2':'B1_HOL',
                              'Rationale 1':'B1_comment',
                              'Analytic 2_1':'B2_TR',
                              'Analytic 2_2':'B2_CC',
                              'Analytic 2_3':'B2_LR',
                              'Analytic 2_4':'B2_GRA',
                              'Holistic 2_1':'B2_HOL',
                              'Rationale 2':'B2_comment',
                              'Analytic 3_1':'C1_TR',
                              'Analytic 3_2':'C1_CC',
                              'Analytic 3_3':'C1_LR',
                              'Analytic 3_4':'C1_GRA',
                              'Holistic 3_1':'C1_HOL',
                              'Rationale 3':'C1_comment',
                              'Gender':'gender',
                              'Age':'age',                 
                              'Education':'education',
                              'Education_6_TEXT':'education_other',
                              'TESOL cert':'TESOL_cert',
                              'TESOL cert_6_TEXT':'TESOL_cert_other',
                              'ELT experience':'ELT_exp_yrs',
                              'IELTS status':'IELTS_status',
                              'IELTS status_8_TEXT':'IELTS_status_other',
                              'IELTS experience':'IELTS_exp_yrs',
                              'L1':'rater_L1',
                              'Student L1':'student_L1',
                              'FL_148_DO':'text_order',
                              'B1texts_DO':'B1_text_id',
                              'B2texts_DO':'B2_text_id',
                              'C1texts_DO':'C1_text_id'}) 

In [5]:
# Convert analytic scores to same scale as holistic

analytic_dict = {'1-':1.0,
                 '1':1.3,
                 '1+':1.7,
                 '2-':2.0,
                 '2':2.3,
                 '2+':2.7,
                 '3-':3.0,
                 '3':3.3,
                 '3+':3.7,
                 '4-':4.0,
                 '4':4.3,
                 '4+':4.7,
                 '5-':5.0,
                 '5':5.3,
                 '5+':5.7,
                 '6-':6.0,
                 '6':6.3,
                 '6+':6.7,
                 '7-':7.0,
                 '7':7.3,
                 '7+':7.7,
                 '8-':8.0,
                 '8':8.3,
                 '8+':8.7,
                 '9-':9.0,
                 '9':9.3,
                 '9+':9.7}

pilot[['B1_TR','B1_CC','B1_LR','B1_GRA','B2_TR','B2_CC','B2_LR','B2_GRA','C1_TR','C1_CC','C1_LR','C1_GRA']] = pilot[['B1_TR','B1_CC','B1_LR','B1_GRA','B2_TR','B2_CC','B2_LR','B2_GRA','C1_TR','C1_CC','C1_LR','C1_GRA']].applymap(analytic_dict.get)

In [6]:
# Convert holistic scores to same scale as main survey (1 to 9.9)

pilot[['B1_HOL','B2_HOL','C1_HOL']] = pilot[['B1_HOL','B2_HOL','C1_HOL']].astype(int)/10

In [7]:
# Add rater_ids

pilot['rater_id'] = ['R' + str(x + 1) for x in pilot.index]

In [8]:
# Deal with 'other' text columns - only one 'other'

pilot.iloc[5,:]

# Replace 'Other' with 'IELTS Examiner Trainer' based on personal knowledge

pilot.loc[(pilot.IELTS_status == 'Other'),'IELTS_status'] = 'IELTS Examiner Trainer'

# Remove unnecessary 'other' columns

del pilot['education_other']
del pilot['TESOL_cert_other']
del pilot['IELTS_status_other']

duration_sec                                                       1357
B1_TR                                                               4.3
B1_CC                                                               5.3
B1_LR                                                               5.3
B1_GRA                                                              4.3
B1_HOL                                                              4.5
B1_comment            Poor use of collocations. No major spelling is...
B2_TR                                                               6.3
B2_CC                                                               6.3
B2_LR                                                               6.3
B2_GRA                                                              6.3
B2_HOL                                                              6.0
B2_comment            Many attempts to use sophisticated lexical chu...
C1_TR                                                           

In [9]:
# Standardize rater_L1 and student_L1 columns

pilot.rater_L1.value_counts()

pilot.student_L1.value_counts()

English              6
Spanish              2
English              1
Pashtu               1
Chinese + English    1
Name: rater_L1, dtype: int64

Arabic                                                            3
Spanish                                                           2
I taught and examined mostly Italian students                     1
Korean                                                            1
Chinese                                                           1
Mixed                                                             1
Spanish, Chinese, Portuguese, Korean, Arabic, Japanese, German    1
Japanese                                                          1
Name: student_L1, dtype: int64

In [10]:
L1_dict = {'Chinese + English':'Chinese; English',
           'English ':'English',
           'I taught and examined mostly Italian students':'Italian',
           'Spanish, Chinese, Portuguese, Korean, Arabic, Japanese, German':'various',
           'Mixed ':'various'
          }

In [11]:
pilot.rater_L1 = pilot.rater_L1.map(L1_dict).fillna(pilot.rater_L1)
pilot.student_L1 = pilot.student_L1.map(L1_dict).fillna(pilot.student_L1)

In [12]:
# Standardize text_order column

text_order_dict = {'FL_144|FL_146|FL_147':'B1_B2_C1',
                  'FL_144|FL_147|FL_146':'B1_C1_B2',
                  'FL_146|FL_144|FL_147':'B2_B1_C1',
                  'FL_146|FL_147|FL_144':'B2_C1_B1',
                  'FL_147|FL_144|FL_146':'C1_B1_B2',
                  'FL_147|FL_146|FL_144':'C1_B2_B1'}

pilot.text_order = pilot.text_order.map(text_order_dict)

In [13]:
# Add additional question from main survey (pilot candidates asked directly)

#Which description best fits your examining experience?
    #Test takers with mostly the same L1 and a narrow range of proficiency levels
    #Test takers with mostly the same L1 but a range of proficiency levels
    #Test takers with a variety of L1s but a narrow range of proficiency levels
    #Test takers with a variety of L1s and a range of proficiency levels
    
#Abbreviated options:
    # same_L1_same_prof
    # same_L1_mix_prof
    # mix_L1_same_prof
    # mix_L1_mix_prof
    
exam_context_dict ={'R1':'mix_L1_mix_prof',
                    'R2':'same_L1_same_prof',
                    'R3':'same_L1_mix_prof',
                    'R4':'mix_L1_mix_prof',
                    'R5':'mix_L1_mix_prof',
                    'R6':'mix_L1_mix_prof',
                    'R7':'same_L1_mix_prof',
                    'R8':'same_L1_same_prof',
                    'R9':'mix_L1_mix_prof',
                    'R10':'mix_L1_mix_prof',
                    'R11':'mix_L1_mix_prof'}

pilot['exam_context'] = pilot.rater_id.map(exam_context_dict)

In [14]:
# Reorder columns

pilot = pilot[['rater_id','B1_TR', 'B1_CC', 'B1_LR', 'B1_GRA', 'B1_HOL',
               'B2_TR', 'B2_CC', 'B2_LR', 'B2_GRA', 'B2_HOL',
               'C1_TR', 'C1_CC', 'C1_LR', 'C1_GRA', 'C1_HOL',
               'duration_sec','text_order','B1_text_id', 'B2_text_id', 'C1_text_id',
               'B1_comment','B2_comment','C1_comment',
               'gender', 'age', 'education', 'TESOL_cert', 'ELT_exp_yrs',
               'IELTS_status', 'IELTS_exp_yrs', 'rater_L1', 'student_L1','exam_context']]
pilot.head(3)

,rater_id,B1_TR,B1_CC,B1_LR,B1_GRA,B1_HOL,B2_TR,B2_CC,B2_LR,B2_GRA,B2_HOL,C1_TR,C1_CC,C1_LR,C1_GRA,C1_HOL,duration_sec,text_order,B1_text_id,B2_text_id,C1_text_id,B1_comment,B2_comment,C1_comment,gender,age,education,TESOL_cert,ELT_exp_yrs,IELTS_status,IELTS_exp_yrs,rater_L1,student_L1,exam_context
0,R1,4.3,4.3,4.3,3.7,3.4,5.3,6.3,6.7,6.7,6.3,7.3,6.3,8.0,7.3,7.6,2499,C1_B1_B2,Text1,Text15,Text23,The candidate uses only basic vocabulary repet...,The candidate takes risks and attempts to use ...,The candidate uses a wide range of vocabulary ...,Female,30-39,Bachelor's degree,Cambridge Delta,6-10,"Former IELTS writing examiner,Former IELTS spe...",6-10,Spanish,Arabic,mix_L1_mix_prof
1,R2,5.0,4.3,4.3,3.7,3.7,7.3,6.7,7.7,7.3,7.8,8.7,8.3,8.7,8.3,9.0,2439,B1_C1_B2,Text3,Text16,Text26,There were a few flashes of less common langua...,"There's a good range, including less common it...","They're clearly very strong, a wide range of l...",Male,30-39,Master's degree,"Cambridge CELTA,Cambridge Delta",11-20,"Former IELTS writing examiner,Former IELTS spe...",3-5,English,Italian,same_L1_same_prof
2,R3,5.3,4.3,5.3,4.3,4.4,5.3,5.7,6.3,6.3,5.6,8.3,8.3,8.3,8.3,8.8,1111,B2_C1_B1,Text9,Text12,Text33,Just sufficient for the task. Some error.,Some good original lexis but relies heavily on...,Precise and wide resource. Very good colocatio...,Prefer not to say,60-69,Master's degree,Trinity CertTESOL,20+,IELTS Examiner Trainer,11-20,English,Spanish,same_L1_mix_prof


## Survey data
In this public version of the data, irrelevant columns and columns including private data (names, emails, etc.) have been removed.

In [15]:
# Read in main survey data (exported from Qualtrics)

survey = pd.read_csv("../docs/survey_data.csv")
survey.head()

,Duration (in seconds),Analytic 1_1,Analytic 1_2,Analytic 1_3,Analytic 1_4,Holistic 1_1,Rationale 1,Analytic 2_1,Analytic 2_2,Analytic 2_3,Analytic 2_4,Holistic 2_1,Rationale 2,Analytic 3_1,Analytic 3_2,Analytic 3_3,Analytic 3_4,Holistic 3_1,Rationale 3,Gender,Age,Education,Education_6_TEXT,TESOL cert,TESOL cert_6_TEXT,ELT experience,IELTS status,IELTS status_8_TEXT,IELTS experience,L1,Student L1,Examining contexts,FL_148_DO,B1texts_DO,B2texts_DO,C1texts_DO
0,955,6-,5-,6,4+,5.8,The candidate used some higher-level lexis som...,6+,6,6+,5+,6.1,Strong use of verb phrases and knowledge of co...,6+,8,8,8,7.8,Fluent and accurate use of higher-level lexis ...,Male,40-49,Master's degree,NaN,Other,MA TESOL,11-20,"Former IELTS writing examiner,Current IELTS sp...",NaN,3-5,English,Arabic,Test takers with mostly the same L1 and a narr...,FL_144|FL_147|FL_146,Text10,Text32,Text28
1,1131,4,4-,5,5,5.0,I noticed regurgitated phrases that were somet...,7,7,6+,6-,6.0,I noticed that the vocabulary used was adequat...,7,7,7+,6,6.5,I noticed good collocation across whole senten...,Female,60-69,Master's degree,NaN,Other,University of Pittsburgh,20+,Former IELTS writing examiner,NaN,3-5,English,Japanese/Arabic/Spanish,Test takers with mostly the same L1 and a narr...,FL_144|FL_147|FL_146,Text8,Text19,Text26
2,2411,5,5+,6-,5,5.5,'- Uses an adequate range of vocabulary for th...,7,7,7,7,7.0,'- uses a sufficient range of vocabulary to al...,8,8-,8+,8,8.1,'- Uses a wide range of vocabulary fluently an...,Male,50-59,Doctorate degree,NaN,"Cambridge CELTA,Cambridge Delta",NaN,20+,"Former IELTS writing examiner,Current IELTS sp...",NaN,11-20,English,Arabic,Test takers with a variety of L1s and a range ...,FL_147|FL_144|FL_146,Text10,Text12,Text23
3,26892,5-,5,5,5-,5.0,There is a limited range of vocabulary. In pa...,7,6,6,6+,6.3,Words like 'endorsed' are used with some aware...,7+,7+,7+,7,7.4,There are some inaccuracies in collocation. T...,Male,40-49,Master's degree,NaN,Other,MA in TESOL with IELTS training and formal tra...,20+,Former IELTS writing examiner,NaN,11-20,English,"Now - Arabic. Formerly, Mandarin, Japanese, K...",Test takers with a variety of L1s and a range ...,FL_146|FL_144|FL_147,Text5,Text14,Text22
4,2920,4,5-,5,4,4.5,did not reach level 6 (adequate range of vocab...,6,5,6,5,5.5,attempts to use less common vocabulary but wit...,8-,8-,8,8,7.8,uses a wide range of vocabulary fluently and f...,Female,40-49,Master's degree,NaN,Other,ESL Specialist Cert,6-10,"Former IELTS writing examiner,Current IELTS sp...",NaN,6-10,English,"Arabic, Chinese; IELTS=Indian languages",Test takers with mostly the same L1 but a rang...,FL_146|FL_147|FL_144,Text9,Text17,Text27


In [16]:
# Rename columns

survey = survey.rename(columns={'Duration (in seconds)':'duration_sec',
                              'Analytic 1_1':'B1_TR',
                              'Analytic 1_2':'B1_CC',
                              'Analytic 1_3':'B1_LR',
                              'Analytic 1_4':'B1_GRA',
                              'Holistic 1_1':'B1_HOL',
                              'Rationale 1':'B1_comment',
                              'Analytic 2_1':'B2_TR',
                              'Analytic 2_2':'B2_CC',
                              'Analytic 2_3':'B2_LR',
                              'Analytic 2_4':'B2_GRA',
                              'Holistic 2_1':'B2_HOL',
                              'Rationale 2':'B2_comment',
                              'Analytic 3_1':'C1_TR',
                              'Analytic 3_2':'C1_CC',
                              'Analytic 3_3':'C1_LR',
                              'Analytic 3_4':'C1_GRA',
                              'Holistic 3_1':'C1_HOL',
                              'Rationale 3':'C1_comment',
                              'Gender':'gender',
                              'Age':'age',                 
                              'Education':'education',
                              'Education_6_TEXT':'education_other',
                              'TESOL cert':'TESOL_cert',
                              'TESOL cert_6_TEXT':'TESOL_cert_other',
                              'ELT experience':'ELT_exp_yrs',
                              'IELTS status':'IELTS_status',
                              'IELTS status_8_TEXT':'IELTS_status_other',
                              'IELTS experience':'IELTS_exp_yrs',
                              'L1':'rater_L1',
                              'Student L1':'student_L1',
                              'Examining contexts':'exam_context',
                              'FL_148_DO':'text_order',
                              'B1texts_DO':'B1_text_id',
                              'B2texts_DO':'B2_text_id',
                              'C1texts_DO':'C1_text_id'}) 

In [17]:
# Convert analytic scores to same scale as holistic

survey[['B1_TR','B1_CC','B1_LR','B1_GRA','B2_TR','B2_CC','B2_LR','B2_GRA','C1_TR','C1_CC','C1_LR','C1_GRA']] = survey[['B1_TR','B1_CC','B1_LR','B1_GRA','B2_TR','B2_CC','B2_LR','B2_GRA','C1_TR','C1_CC','C1_LR','C1_GRA']].applymap(analytic_dict.get)

In [18]:
# Convert holistic scores to float

survey[['B1_HOL','B2_HOL','C1_HOL']] = survey[['B1_HOL','B2_HOL','C1_HOL']].astype(float)

In [19]:
# Add rater_ids

survey['rater_id'] = ['R' + str(x + 12) for x in survey.index]

In [20]:
# Standardize'Other' responses

survey.education_other.value_counts()

Series([], Name: education_other, dtype: int64)

In [21]:
survey.TESOL_cert_other.value_counts()

MA TESOL                                                                    2
University of Pittsburgh                                                    1
MA in TESOL with IELTS training and formal training in technical writing    1
ESL Specialist Cert                                                         1
CEELT 1 and 2                                                               1
Name: TESOL_cert_other, dtype: int64

In [22]:
survey.loc[(survey.TESOL_cert_other == 'CEELT 1 and 2'),'TESOL_cert_other'] = 'CEELT 2'
survey.loc[(survey.TESOL_cert_other == 'University of Pittsburgh'),'TESOL_cert_other'] = 'MA TESOL'
survey.loc[(survey.TESOL_cert_other == 'MA in TESOL with IELTS training and formal training in technical writing'),'TESOL_cert_other'] = 'MA TESOL'
survey.TESOL_cert_other.value_counts()

MA TESOL               4
ESL Specialist Cert    1
CEELT 2                1
Name: TESOL_cert_other, dtype: int64

In [23]:
# Combine TESOL_cert columns

survey.TESOL_cert = [x if x != 'Other' else np.nan for x in survey.TESOL_cert]
survey.TESOL_cert = survey.TESOL_cert.fillna(survey.TESOL_cert_other)
survey.TESOL_cert = survey.TESOL_cert.fillna('Not_stated')

In [24]:
survey.IELTS_status_other.value_counts()

Series([], Name: IELTS_status_other, dtype: int64)

In [25]:
# Remove unnecessary 'other' columns

del survey['education_other']
del survey['TESOL_cert_other']
del survey['IELTS_status_other']

In [26]:
# Standardize rater_L1 and student_L1 columns

survey.rater_L1.value_counts()
survey.student_L1.value_counts()

English       31
Portuguese     3
English        1
Polish         1
Eng/Rus        1
Name: rater_L1, dtype: int64

Arabic                                                                                                                               12
Portuguese                                                                                                                            2
Spanish                                                                                                                               2
All languages                                                                                                                         1
Heb/Rus/Ara                                                                                                                           1
Argentine Spanish                                                                                                                     1
English                                                                                                                               1
hard to say, they're from all over the world    

In [27]:
# Update L1 dict

L1_dict = {'Chinese + English':'Chinese; English',
           'English ':'English',
           'Eng/Rus':'English; Russian',
           'I taught and examined mostly Italian students':'Italian',
           'Spanish, Chinese, Portuguese, Korean, Arabic, Japanese, German':'various',
           'Mixed':'various',
           'Arabic--when I was an IELTS examiner':'Arabic',
           'a variety of Indian languages and Arabic':'various',
           'All languages':'various',
           'Now - Arabic.  Formerly, Mandarin, Japanese, Korean, and Spanish - at different points in my career.':'various',
           'Heb/Rus/Ara':'various',  
           'Arabic, Chinese; IELTS=Indian languages':'various',
           'Japanese/Arabic/Spanish':'various',
           'Multilingual':'various',
           'A variety: Polish, Spanish, Hindi':'various',
           'Arabic/Korean/Turkish':'various',
           'Arabic before':'Arabic',
           'Mandarin Chinese':'Chinese',
           "hard to say, they're from all over the world":'various',
           'German/Spanish':'various',
           "I've examined students in three different countries and the common language has been Spanish, Tunisian Arabic, French and Bangla.":'various',
           'Argentine Spanish':'Spanish'
          }

In [28]:
survey.rater_L1 = survey.rater_L1.map(L1_dict).fillna(survey.rater_L1)
survey.student_L1 = survey.student_L1.map(L1_dict).fillna(survey.student_L1)

In [29]:
survey.rater_L1.value_counts()
survey.student_L1.value_counts()

English             32
Portuguese           3
Polish               1
English; Russian     1
Name: rater_L1, dtype: int64

Arabic        14
various       12
Spanish        3
Portuguese     2
Chinese        2
Punjabi        1
Sinhalese      1
English        1
Hebrew         1
Name: student_L1, dtype: int64

In [30]:
# Standardize text_order column

text_order_dict = {'FL_144|FL_146|FL_147':'B1_B2_C1',
                  'FL_144|FL_147|FL_146':'B1_C1_B2',
                  'FL_146|FL_144|FL_147':'B2_B1_C1',
                  'FL_146|FL_147|FL_144':'B2_C1_B1',
                  'FL_147|FL_144|FL_146':'C1_B1_B2',
                  'FL_147|FL_146|FL_144':'C1_B2_B1'}

survey.text_order = survey.text_order.map(text_order_dict)

In [31]:
# Shorten exam_context answers
    
exam_context_dict2 ={'Test takers with a variety of L1s and a range of proficiency levels':'mix_L1_mix_prof',
                     'Test takers with mostly the same L1 and a narrow range of proficiency levels':'same_L1_same_prof',
                     'Test takers with mostly the same L1 but a range of proficiency levels':'same_L1_mix_prof',
                     'Test takers with a variety of L1s but  a narrow range of proficiency levels':'mix_L1_same_prof'}

survey['exam_context'] = survey.exam_context.map(exam_context_dict2)

In [32]:
survey.exam_context.value_counts()

mix_L1_mix_prof      16
same_L1_mix_prof      9
same_L1_same_prof     7
mix_L1_same_prof      5
Name: exam_context, dtype: int64

In [33]:
# Reorder columns

survey = survey[['rater_id','B1_TR', 'B1_CC', 'B1_LR', 'B1_GRA', 'B1_HOL',
               'B2_TR', 'B2_CC', 'B2_LR', 'B2_GRA', 'B2_HOL',
               'C1_TR', 'C1_CC', 'C1_LR', 'C1_GRA', 'C1_HOL',
               'duration_sec','text_order','B1_text_id', 'B2_text_id', 'C1_text_id',
               'B1_comment','B2_comment','C1_comment',
               'gender', 'age', 'education', 'TESOL_cert', 'ELT_exp_yrs',
               'IELTS_status', 'IELTS_exp_yrs', 'rater_L1', 'student_L1','exam_context']]
survey.head()

,rater_id,B1_TR,B1_CC,B1_LR,B1_GRA,B1_HOL,B2_TR,B2_CC,B2_LR,B2_GRA,B2_HOL,C1_TR,C1_CC,C1_LR,C1_GRA,C1_HOL,duration_sec,text_order,B1_text_id,B2_text_id,C1_text_id,B1_comment,B2_comment,C1_comment,gender,age,education,TESOL_cert,ELT_exp_yrs,IELTS_status,IELTS_exp_yrs,rater_L1,student_L1,exam_context
0,R12,6.0,5.0,6.3,4.7,5.8,6.7,6.3,6.7,5.7,6.1,6.7,8.3,8.3,8.3,7.8,955,B1_C1_B2,Text10,Text32,Text28,The candidate used some higher-level lexis som...,Strong use of verb phrases and knowledge of co...,Fluent and accurate use of higher-level lexis ...,Male,40-49,Master's degree,MA TESOL,11-20,"Former IELTS writing examiner,Current IELTS sp...",3-5,English,Arabic,same_L1_same_prof
1,R13,4.3,4.0,5.3,5.3,5.0,7.3,7.3,6.7,6.0,6.0,7.3,7.3,7.7,6.3,6.5,1131,B1_C1_B2,Text8,Text19,Text26,I noticed regurgitated phrases that were somet...,I noticed that the vocabulary used was adequat...,I noticed good collocation across whole senten...,Female,60-69,Master's degree,MA TESOL,20+,Former IELTS writing examiner,3-5,English,various,same_L1_same_prof
2,R14,5.3,5.7,6.0,5.3,5.5,7.3,7.3,7.3,7.3,7.0,8.3,8.0,8.7,8.3,8.1,2411,C1_B1_B2,Text10,Text12,Text23,'- Uses an adequate range of vocabulary for th...,'- uses a sufficient range of vocabulary to al...,'- Uses a wide range of vocabulary fluently an...,Male,50-59,Doctorate degree,"Cambridge CELTA,Cambridge Delta",20+,"Former IELTS writing examiner,Current IELTS sp...",11-20,English,Arabic,mix_L1_mix_prof
3,R15,5.0,5.3,5.3,5.0,5.0,7.3,6.3,6.3,6.7,6.3,7.7,7.7,7.7,7.3,7.4,26892,B2_B1_C1,Text5,Text14,Text22,There is a limited range of vocabulary. In pa...,Words like 'endorsed' are used with some aware...,There are some inaccuracies in collocation. T...,Male,40-49,Master's degree,MA TESOL,20+,Former IELTS writing examiner,11-20,English,various,mix_L1_mix_prof
4,R16,4.3,5.0,5.3,4.3,4.5,6.3,5.3,6.3,5.3,5.5,8.0,8.0,8.3,8.3,7.8,2920,B2_C1_B1,Text9,Text17,Text27,did not reach level 6 (adequate range of vocab...,attempts to use less common vocabulary but wit...,uses a wide range of vocabulary fluently and f...,Female,40-49,Master's degree,ESL Specialist Cert,6-10,"Former IELTS writing examiner,Current IELTS sp...",6-10,English,various,same_L1_mix_prof


## Final dataframe

In [34]:
# Combine dataframes

ratings = pd.concat([pilot,survey]).reset_index(drop=True)
ratings.head()

,rater_id,B1_TR,B1_CC,B1_LR,B1_GRA,B1_HOL,B2_TR,B2_CC,B2_LR,B2_GRA,B2_HOL,C1_TR,C1_CC,C1_LR,C1_GRA,C1_HOL,duration_sec,text_order,B1_text_id,B2_text_id,C1_text_id,B1_comment,B2_comment,C1_comment,gender,age,education,TESOL_cert,ELT_exp_yrs,IELTS_status,IELTS_exp_yrs,rater_L1,student_L1,exam_context
0,R1,4.3,4.3,4.3,3.7,3.4,5.3,6.3,6.7,6.7,6.3,7.3,6.3,8.0,7.3,7.6,2499,C1_B1_B2,Text1,Text15,Text23,The candidate uses only basic vocabulary repet...,The candidate takes risks and attempts to use ...,The candidate uses a wide range of vocabulary ...,Female,30-39,Bachelor's degree,Cambridge Delta,6-10,"Former IELTS writing examiner,Former IELTS spe...",6-10,Spanish,Arabic,mix_L1_mix_prof
1,R2,5.0,4.3,4.3,3.7,3.7,7.3,6.7,7.7,7.3,7.8,8.7,8.3,8.7,8.3,9.0,2439,B1_C1_B2,Text3,Text16,Text26,There were a few flashes of less common langua...,"There's a good range, including less common it...","They're clearly very strong, a wide range of l...",Male,30-39,Master's degree,"Cambridge CELTA,Cambridge Delta",11-20,"Former IELTS writing examiner,Former IELTS spe...",3-5,English,Italian,same_L1_same_prof
2,R3,5.3,4.3,5.3,4.3,4.4,5.3,5.7,6.3,6.3,5.6,8.3,8.3,8.3,8.3,8.8,1111,B2_C1_B1,Text9,Text12,Text33,Just sufficient for the task. Some error.,Some good original lexis but relies heavily on...,Precise and wide resource. Very good colocatio...,Prefer not to say,60-69,Master's degree,Trinity CertTESOL,20+,IELTS Examiner Trainer,11-20,English,Spanish,same_L1_mix_prof
3,R4,4.0,4.3,4.0,4.0,3.9,7.3,7.3,7.0,7.0,6.5,8.3,8.3,8.3,8.3,8.1,2068,B1_B2_C1,Text5,Text13,Text29,basic vocabulary\nsome items not related to ta...,sufficient range to allow some flexibility - d...,uses a wide range to covey precise meaning - p...,Male,50-59,Master's degree,"Cambridge CELTA,Cambridge Delta",20+,"Current IELTS writing examiner,Current IELTS s...",11-20,English,Korean,mix_L1_mix_prof
4,R5,4.3,5.3,5.3,5.0,4.5,7.3,6.0,6.3,6.0,5.4,8.3,6.3,8.7,7.0,7.5,4534,C1_B1_B2,Text6,Text11,Text27,The effect of error on communication was high ...,Attempts to use less common vocabulary but thi...,Precision of meaning; skillful use of less com...,Female,60-69,Master's degree,"Cambridge CELTA,Cambridge Delta",20+,"Former IELTS writing examiner,Current IELTS sp...",20+,English,Chinese,mix_L1_mix_prof


#### Standardize TESOL cert values 
- keep only highest qualification

In [35]:
ratings.TESOL_cert.value_counts()

Cambridge CELTA,Cambridge Delta          19
Cambridge Delta                           9
Cambridge CELTA                           8
MA TESOL                                  4
Not_stated                                2
Trinity CertTESOL                         1
ESL Specialist Cert                       1
Cambridge CELTA,Trinity DipTESOL          1
SIT TESOL Certificate                     1
Cambridge CELTA,Cambridge Delta,Other     1
Trinity CertTESOL,Trinity DipTESOL        1
Name: TESOL_cert, dtype: int64

In [36]:
tesol_cert_dict = {'Cambridge CELTA,Cambridge Delta':'Cambridge Delta',
                   'SIT TESOL Certificate':'SIT TESOL Cert',
                   'Trinity CertTESOL,Trinity DipTESOL':'Trinity DipTESOL',
                   'Cambridge CELTA,Trinity DipTESOL':'Trinity DipTESOL',
                   'Cambridge CELTA,Cambridge Delta,Other':'Cambridge Delta'}

ratings.TESOL_cert = ratings.TESOL_cert.map(tesol_cert_dict).fillna(ratings.TESOL_cert)

In [37]:
ratings.TESOL_cert.value_counts()
sum(ratings.TESOL_cert.value_counts())
len(ratings)

Cambridge Delta        29
Cambridge CELTA         8
MA TESOL                4
Trinity DipTESOL        2
Not_stated              2
Trinity CertTESOL       1
ESL Specialist Cert     1
SIT TESOL Cert          1
Name: TESOL_cert, dtype: int64

48

48

#### Standardize IELTS status values 
- create separate 'former/current' column
- keep only highest rank
- cut speaking examiner status

In [38]:
ratings.IELTS_status.value_counts()

Former IELTS writing examiner,Former IELTS speaking examiner                                                   18
Former IELTS writing examiner,Current IELTS speaking examiner                                                  13
Former IELTS writing examiner                                                                                   4
IELTS Examiner Trainer                                                                                          2
Current IELTS writing examiner,Current IELTS speaking examiner                                                  2
Former IELTS writing examiner,Current IELTS speaking examiner,IELTS Examiner Trainer                            2
Former IELTS writing examiner,IELTS Examiner Trainer                                                            2
Current IELTS writing examiner,Current IELTS speaking examiner,IELTS Senior Examiner,IELTS Examiner Trainer     1
Former IELTS writing examiner,Current IELTS speaking examiner,IELTS Senior Examiner,IELT

In [39]:
former_current_dict = {'Former IELTS writing examiner,Former IELTS speaking examiner':'former',
                       'Former IELTS writing examiner,Current IELTS speaking examiner':'former',
                       'Former IELTS writing examiner':'former',
                       'IELTS Examiner Trainer':'current',
                       'Former IELTS writing examiner,Current IELTS speaking examiner,IELTS Examiner Trainer':'current',
                       'Former IELTS writing examiner,Current IELTS speaking examiner,IELTS Senior Examiner,IELTS Examiner Trainer':'current',
                       'Former IELTS writing examiner,Former IELTS speaking examiner,IELTS Examiner Trainer':'former',
                       'Current IELTS speaking examiner':'current',
                       'Current IELTS writing examiner,Current IELTS speaking examiner,IELTS Senior Examiner,IELTS Examiner Trainer':'current',
                       'Current IELTS writing examiner,Current IELTS speaking examiner':'current',
                       'Former IELTS writing examiner,IELTS Examiner Trainer':'former',
                       'Other':'Not an examiner'}


IELTS_status_dict = {'Former IELTS writing examiner,Former IELTS speaking examiner':'examiner',
                     'Former IELTS writing examiner,Current IELTS speaking examiner':'examiner',
                     'Former IELTS writing examiner':'examiner',
                     'IELTS Examiner Trainer':'examiner trainer',
                     'Former IELTS writing examiner,Current IELTS speaking examiner,IELTS Examiner Trainer':'examiner trainer',
                     'Former IELTS writing examiner,Current IELTS speaking examiner,IELTS Senior Examiner,IELTS Examiner Trainer':'senior examiner and trainer',
                     'Former IELTS writing examiner,Former IELTS speaking examiner,IELTS Examiner Trainer':'examiner trainer',
                     'Current IELTS speaking examiner':'examiner', #I know this person is a writing examiner
                     'Current IELTS writing examiner,Current IELTS speaking examiner,IELTS Senior Examiner,IELTS Examiner Trainer':'senior examiner and trainer',
                     'Current IELTS writing examiner,Current IELTS speaking examiner':'examiner',
                     'Former IELTS writing examiner,IELTS Examiner Trainer':'examiner trainer',
                     'Other':'Not an examiner'}

In [40]:
ratings['current'] = ratings.IELTS_status.map(former_current_dict)
ratings.IELTS_status = ratings.IELTS_status.map(IELTS_status_dict)

In [41]:
ratings.IELTS_status.value_counts()
ratings.current.value_counts()

examiner                       38
examiner trainer                7
senior examiner and trainer     2
Not an examiner                 1
Name: IELTS_status, dtype: int64

former             38
current             9
Not an examiner     1
Name: current, dtype: int64

#### Rename education values 

In [42]:
ratings.education.value_counts()

Master's degree      36
Bachelor's degree     8
Doctorate degree      4
Name: education, dtype: int64

In [43]:
education_dict = {"Master's degree":'MA', "Doctorate degree":'PhD',"Bachelor's degree":'BA'}
ratings.education = ratings.education.map(education_dict)
ratings.education.value_counts()

MA     36
BA      8
PhD     4
Name: education, dtype: int64

#### Rename gender values 

In [44]:
ratings.gender.value_counts()

Male                 24
Female               21
Prefer not to say     3
Name: gender, dtype: int64

In [45]:
ratings = ratings.replace('Prefer not to say', 'unknown')

In [46]:
ratings.gender.value_counts()

Male       24
Female     21
unknown     3
Name: gender, dtype: int64

#### Rename exp values 

In [47]:
ratings = ratings.replace('20+', '>20')
ratings = ratings.replace('less than 1', '1>')

#### Create `rater_English` column

In [48]:
ratings.rater_L1.value_counts()

English             39
Portuguese           3
Spanish              2
Pashtu               1
Chinese; English     1
Polish               1
English; Russian     1
Name: rater_L1, dtype: int64

In [49]:
ratings['rater_English'] = ['L1' if x in ['English','English; Russian'] else 'L2' for x in ratings.rater_L1]
ratings.rater_English.value_counts()

L1    40
L2     8
Name: rater_English, dtype: int64

#### Create `L1_range` column

In [50]:
ratings.exam_context.value_counts()

mix_L1_mix_prof      23
same_L1_mix_prof     11
same_L1_same_prof     9
mix_L1_same_prof      5
Name: exam_context, dtype: int64

In [51]:
ratings['L1_range'] = ['wide' if x in ['mix_L1_mix_prof', 'mix_L1_same_prof'] else 'narrow' for x in ratings.exam_context]
ratings.L1_range.value_counts()

wide      28
narrow    20
Name: L1_range, dtype: int64

#### Create `prof_range` column

In [52]:
ratings.exam_context.value_counts()

mix_L1_mix_prof      23
same_L1_mix_prof     11
same_L1_same_prof     9
mix_L1_same_prof      5
Name: exam_context, dtype: int64

In [53]:
ratings['prof_range'] = ['wide' if x in ['mix_L1_mix_prof', 'same_L1_mix_prof'] else 'narrow' for x in ratings.exam_context]
ratings.prof_range.value_counts()

wide      34
narrow    14
Name: prof_range, dtype: int64

#### Reorder columns

In [54]:
ratings = ratings[['rater_id', 'B1_TR', 'B1_CC', 'B1_LR', 'B1_GRA', 'B1_HOL', 'B2_TR',
       'B2_CC', 'B2_LR', 'B2_GRA', 'B2_HOL', 'C1_TR', 'C1_CC', 'C1_LR',
       'C1_GRA', 'C1_HOL', 'duration_sec', 'text_order', 'B1_text_id',
       'B2_text_id', 'C1_text_id', 'B1_comment', 'B2_comment', 'C1_comment',
       'gender', 'age', 'education', 'TESOL_cert', 'ELT_exp_yrs',
       'IELTS_status', 'current', 'IELTS_exp_yrs', 'rater_L1', 'rater_English',
       'student_L1', 'L1_range', 'prof_range']]

In [55]:
# Write out ratings csv and pickle

ratings.to_csv('../docs/ratings.csv',index=False)
joblib.dump(ratings,'../docs/ratings.pkl')

['../docs/ratings.pkl']

#### Long version of data frame
Each response as row, necessary for MFRM analysis

In [56]:
# Copy ratings dataframe
ratings_long = ratings.copy()

# Create list of all text-specific columns
ratings_long['B1'] = list(zip(ratings_long.B1_TR, ratings_long.B1_CC,ratings_long.B1_LR,ratings_long.B1_GRA,
                              ratings_long.B1_HOL,ratings_long.B1_text_id,ratings_long.B1_comment))
ratings_long['B2'] = list(zip(ratings_long.B2_TR, ratings_long.B2_CC,ratings_long.B2_LR,ratings_long.B2_GRA,
                              ratings_long.B2_HOL,ratings_long.B2_text_id,ratings_long.B2_comment))
ratings_long['C1'] = list(zip(ratings_long.C1_TR, ratings_long.C1_CC,ratings_long.C1_LR,ratings_long.C1_GRA,
                              ratings_long.C1_HOL,ratings_long.C1_text_id,ratings_long.C1_comment))

# Create large column with B1, B2, C1 in single list
ratings_long['temp'] = list(zip(ratings_long.B1,ratings_long.B2,ratings_long.C1))

In [57]:
# Keep only necessary columns

ratings_long = ratings_long[['rater_id','duration_sec','text_order','gender','age','education',
                             'TESOL_cert','ELT_exp_yrs','IELTS_status','current','IELTS_exp_yrs',
                             'rater_L1','rater_English','student_L1','L1_range','prof_range','temp']]

In [58]:
# Explode dataframe

ratings_long = ratings_long.explode('temp').reset_index(drop=True)
ratings_long.head()

,rater_id,duration_sec,text_order,gender,age,education,TESOL_cert,ELT_exp_yrs,IELTS_status,current,IELTS_exp_yrs,rater_L1,rater_English,student_L1,L1_range,prof_range,temp
0,R1,2499,C1_B1_B2,Female,30-39,BA,Cambridge Delta,6-10,examiner,former,6-10,Spanish,L2,Arabic,wide,wide,"(4.3, 4.3, 4.3, 3.7, 3.4, Text1, The candidate..."
1,R1,2499,C1_B1_B2,Female,30-39,BA,Cambridge Delta,6-10,examiner,former,6-10,Spanish,L2,Arabic,wide,wide,"(5.3, 6.3, 6.7, 6.7, 6.3, Text15, The candidat..."
2,R1,2499,C1_B1_B2,Female,30-39,BA,Cambridge Delta,6-10,examiner,former,6-10,Spanish,L2,Arabic,wide,wide,"(7.3, 6.3, 8.0, 7.3, 7.6, Text23, The candidat..."
3,R2,2439,B1_C1_B2,Male,30-39,MA,Cambridge Delta,11-20,examiner,former,3-5,English,L1,Italian,narrow,narrow,"(5.0, 4.3, 4.3, 3.7, 3.7, Text3, There were a ..."
4,R2,2439,B1_C1_B2,Male,30-39,MA,Cambridge Delta,11-20,examiner,former,3-5,English,L1,Italian,narrow,narrow,"(7.3, 6.7, 7.7, 7.3, 7.8, Text16, There's a go..."


In [59]:
# Create CEFR column

ratings_long['CEFR'] = ['B1','B2','C1']*len(ratings)

In [60]:
# Re-create text-specific columns using 'temp' column

ratings_long[['TR','CC','LR','GRA','HOL','text_id','comment']] = pd.DataFrame(ratings_long.temp.tolist(), index= ratings_long.index)

In [61]:
# Create a 'response_id' column

ratings_long['response_id'] = ratings_long.index + 1

In [62]:
# Create 'sophistication', 'sophistication type' and 'accuracy' columns

soph_dict = {'Text1':['low',np.nan,'low'],
             'Text2':['low',np.nan,'high'],
             'Text3':['mid','col','low'],
             'Text4':['mid','col','high'],
             'Text5':['mid','non_col','low'],
             'Text6':['mid','non_col','high'],
             'Text7':['high','col','low'],
             'Text8':['high','col','high'],
             'Text9':['high','non_col','low'],
             'Text10':['high','non_col','high'],
             'Text31':[np.nan,np.nan,np.nan],
             'Text11':['low',np.nan,'low'],
             'Text12':['low',np.nan,'high'],
             'Text13':['mid','col','low'],
             'Text14':['mid','col','high'],
             'Text15':['mid','non_col','low'],
             'Text16':['mid','non_col','high'],
             'Text17':['high','col','low'],
             'Text18':['high','col','high'],
             'Text19':['high','non_col','low'],
             'Text20':['high','non_col','high'],
             'Text32':[np.nan,np.nan,np.nan],
             'Text21':['low',np.nan,'low'],
             'Text22':['low',np.nan,'high'],
             'Text23':['mid','col','low'],
             'Text24':['mid','col','high'],
             'Text25':['mid','non_col','low'],
             'Text26':['mid','non_col','high'],
             'Text27':['high','col','low'],
             'Text28':['high','col','high'],
             'Text29':['high','non_col','low'],
             'Text30':['high','non_col','high'],
             'Text33':[np.nan,np.nan,np.nan]}

In [63]:
# Create columns

ratings_long['soph'] = ratings_long.text_id.map(soph_dict).apply(lambda x: x[0])
ratings_long['soph_type'] = ratings_long.text_id.map(soph_dict).apply(lambda x: x[1])
ratings_long['accuracy'] = ratings_long.text_id.map(soph_dict).apply(lambda x: x[2])

In [64]:
# Replace NaN values in soph_type column with 'NA' which is used in R

ratings_long.soph_type = ratings_long.soph_type.fillna('NA')
ratings_long.soph_type.value_counts()

col        54
non_col    53
NA         37
Name: soph_type, dtype: int64

#### Clean up dataframe

In [65]:
# Simplify TESOL_cert categories

ratings_long.TESOL_cert.value_counts()

Cambridge Delta        87
Cambridge CELTA        24
MA TESOL               12
Trinity DipTESOL        6
Not_stated              6
Trinity CertTESOL       3
ESL Specialist Cert     3
SIT TESOL Cert          3
Name: TESOL_cert, dtype: int64

In [66]:
simple_TESOL_cert_dict = {'Cambridge Delta':'diploma',
                          'Cambridge CELTA':'certificate',
                          'MA TESOL':'degree',
                          'Not_stated':'unknown',
                          'Trinity DipTESOL':'diploma',
                          'ESL Specialist Cert':'certificate',
                          'Trinity CertTESOL':'certificate',
                          'SIT TESOL Cert':'certificate'}

ratings_long.TESOL_cert = ratings_long.TESOL_cert.map(simple_TESOL_cert_dict)

In [67]:
# Convert duration to minutes so scales not so uneven

ratings_long.duration_sec = (ratings_long.duration_sec.astype(int)/60).round(1)
ratings_long = ratings_long.rename(columns={"duration_sec": "duration"})

In [68]:
# Simplify duration - bucket times (quantiles)

ratings_long['duration'] = pd.qcut(ratings_long['duration'], q=4).astype(str)
ratings_long.duration.value_counts()

(40.4, 89.8]       36
(4.999, 21.9]      36
(21.9, 40.4]       36
(89.8, 18995.6]    36
Name: duration, dtype: int64

In [69]:
sorted(set(ratings_long.duration))

['(21.9, 40.4]', '(4.999, 21.9]', '(40.4, 89.8]', '(89.8, 18995.6]']

In [70]:
# rename duration column

duration_dict = {sorted(set(ratings_long.duration))[1]:'q1',
                 sorted(set(ratings_long.duration))[0]:'q2',
                 sorted(set(ratings_long.duration))[2]:'q3',
                 sorted(set(ratings_long.duration))[3]:'q4'}
ratings_long.duration = ratings_long.duration.map(duration_dict)

In [71]:
# Re-order columns

ratings_long = ratings_long[['response_id','text_id','rater_id', 'CEFR','TR','CC', 'LR', 'GRA','HOL',
                             'soph','soph_type','accuracy',
                             'comment','duration','text_order','gender','age','education','TESOL_cert',
                             'ELT_exp_yrs','IELTS_status','current','IELTS_exp_yrs',
                             'rater_L1','rater_English','student_L1','L1_range','prof_range']]

In [72]:
ratings_long.head()

,response_id,text_id,rater_id,CEFR,TR,CC,LR,GRA,HOL,soph,soph_type,accuracy,comment,duration,text_order,gender,age,education,TESOL_cert,ELT_exp_yrs,IELTS_status,current,IELTS_exp_yrs,rater_L1,rater_English,student_L1,L1_range,prof_range
0,1,Text1,R1,B1,4.3,4.3,4.3,3.7,3.4,low,NA,low,The candidate uses only basic vocabulary repet...,q3,C1_B1_B2,Female,30-39,BA,diploma,6-10,examiner,former,6-10,Spanish,L2,Arabic,wide,wide
1,2,Text15,R1,B2,5.3,6.3,6.7,6.7,6.3,mid,non_col,low,The candidate takes risks and attempts to use ...,q3,C1_B1_B2,Female,30-39,BA,diploma,6-10,examiner,former,6-10,Spanish,L2,Arabic,wide,wide
2,3,Text23,R1,C1,7.3,6.3,8.0,7.3,7.6,mid,col,low,The candidate uses a wide range of vocabulary ...,q3,C1_B1_B2,Female,30-39,BA,diploma,6-10,examiner,former,6-10,Spanish,L2,Arabic,wide,wide
3,4,Text3,R2,B1,5.0,4.3,4.3,3.7,3.7,mid,col,low,There were a few flashes of less common langua...,q3,B1_C1_B2,Male,30-39,MA,diploma,11-20,examiner,former,3-5,English,L1,Italian,narrow,narrow
4,5,Text16,R2,B2,7.3,6.7,7.7,7.3,7.8,mid,non_col,high,"There's a good range, including less common it...",q3,B1_C1_B2,Male,30-39,MA,diploma,11-20,examiner,former,3-5,English,L1,Italian,narrow,narrow


## Outlier

Individual rater R41 - Not an IELTS examiner (wrote 'other' in examiner status field), anonymous, less than 1 year exp (i.e., none!). Further removal of outliers will occur at the statistical analysis stage.

In [73]:
# Remove R41

ratings_long = ratings_long.loc[ratings_long.rater_id != 'R41']
len(ratings_long)

141

In [74]:
# Write out ratings_long

joblib.dump(ratings_long,'../docs/ratings_long.pkl')

['../docs/ratings_long.pkl']

## MFRM preparation

Prepare file for use with the FACETS software.

In [75]:
# Start with long ratings

mfrm = ratings_long.copy()

In [76]:
# Create temp column containing all ratings and their band, then explode so each one is a new column

mfrm['temp'] =  list(zip(mfrm.TR,mfrm.CC,mfrm.LR,mfrm.GRA,mfrm.HOL))
mfrm['temp'] = [(('TR',x[0]),('CC',x[1]),('LR',x[2]),('GRA',x[3]),('HOL',x[4])) for x in mfrm['temp']]
mfrm = mfrm.explode('temp')
mfrm['band'] = [x[0] for x in mfrm.temp]
mfrm['score'] = [x[1] for x in mfrm.temp]

In [77]:
# Remove columns that won't be used in mfrm analysis

mfrm.columns

Index(['response_id', 'text_id', 'rater_id', 'CEFR', 'TR', 'CC', 'LR', 'GRA',
       'HOL', 'soph', 'soph_type', 'accuracy', 'comment', 'duration',
       'text_order', 'gender', 'age', 'education', 'TESOL_cert', 'ELT_exp_yrs',
       'IELTS_status', 'current', 'IELTS_exp_yrs', 'rater_L1', 'rater_English',
       'student_L1', 'L1_range', 'prof_range', 'temp', 'band', 'score'],
      dtype='object')

In [78]:
# Reorder columns

mfrm = mfrm[['text_id', 'rater_id','band','score','duration','text_order', 'gender', 'age', 'education', 'TESOL_cert',
             'ELT_exp_yrs','IELTS_status', 'current', 'IELTS_exp_yrs','rater_English', 'L1_range', 'prof_range']].reset_index(drop=True)

In [79]:
# Convert score scale to integer (multiply by 10 to remove decimal)

mfrm.score = (mfrm.score*10).astype(int)
mfrm.head()

,text_id,rater_id,band,score,duration,text_order,gender,age,education,TESOL_cert,ELT_exp_yrs,IELTS_status,current,IELTS_exp_yrs,rater_English,L1_range,prof_range
0,Text1,R1,TR,43,q3,C1_B1_B2,Female,30-39,BA,diploma,6-10,examiner,former,6-10,L2,wide,wide
1,Text1,R1,CC,43,q3,C1_B1_B2,Female,30-39,BA,diploma,6-10,examiner,former,6-10,L2,wide,wide
2,Text1,R1,LR,43,q3,C1_B1_B2,Female,30-39,BA,diploma,6-10,examiner,former,6-10,L2,wide,wide
3,Text1,R1,GRA,37,q3,C1_B1_B2,Female,30-39,BA,diploma,6-10,examiner,former,6-10,L2,wide,wide
4,Text1,R1,HOL,34,q3,C1_B1_B2,Female,30-39,BA,diploma,6-10,examiner,former,6-10,L2,wide,wide


### Convert 'facets' categorical variables to integer labels for Facets software

**Key:**
- text_id: 1-30
- rater_id: 1-... (depending on data size)
- band: TR=1, CC=2, LR=3, GRA=4, HOL=5
- duration: (4.999, 22.2]:1, (22.2, 41.1]:2, (41.1, 102.4]:3, (102.4, 18995.6]:4
- gender: Male=1, Female=2, Prefer not to say=M
- age: 30-39:1, 40-49:2, 50-59:3, 60-69:4
- education: BA:1, MA:2, PhD:3
- TESOL_cert: certificate=1, diploma=2, degree=3, unknown=M
- ELT_exp_yrs: 6-10=1, 11-20=2, >20=3
- IELTS_status: examiner=1, examiner trainer=2, senior examiner and trainer=3
- current: current=1, former=2
- IELTS_exp_yrs: >1=1, 1-2=2, 3-5=3, 6-10=4, 11=20=5, >20=6
- rater_English: L1=1, L2=2
- L1_range: narrow=1, wide=2
- prof_range: narrow=1, wide=2

*Code for all missing data is M

In [80]:
# text_id column

mfrm.text_id = [x[4:] for x in mfrm.text_id]

In [81]:
# rater_id column

mfrm.rater_id = [x[1:] for x in mfrm.rater_id]

In [82]:
# band column

band_dict = {'TR':'1', 'CC':'2', 'LR':'3', 'GRA':'4', 'HOL':'5'}
mfrm.band = mfrm.band.map(band_dict)

In [83]:
# duration column

duration_dict = {'q1':1, 'q2':2, 'q3':3, 'q4':4}
mfrm.duration = mfrm.duration.map(duration_dict)

In [84]:
# gender column

gender_dict = {'Male':'1', 'Female':'2', 'Prefer not to say':'M'}
mfrm.gender = mfrm.gender.map(gender_dict)

In [85]:
# age column

age_dict = {'30-39':'1', '40-49':'2', '50-59':'3', '60-69':'4'}
mfrm.age = mfrm.age.map(age_dict)

In [86]:
# education column

education_dict = {'BA':'1', 'MA':'2', 'PhD':'3'}
mfrm.education = mfrm.education.map(education_dict)

In [87]:
# TESOL_cert column

TESOL_cert_dict = {'certificate':'1', 'diploma':'2', 'degree':'3', 'unknown':'M'}
mfrm.TESOL_cert = mfrm.TESOL_cert.map(TESOL_cert_dict)

In [88]:
# ELT_exp column

ELT_exp_yrs_dict = {'6-10':'1', '11-20':'2', '>20':'3'}
mfrm.ELT_exp_yrs = mfrm.ELT_exp_yrs.map(ELT_exp_yrs_dict)

In [89]:
# IELTS_status column

IELTS_status_dict = {'examiner':'1', 'examiner trainer':'2', 'senior examiner and trainer':'3'}
mfrm.IELTS_status = mfrm.IELTS_status.map(IELTS_status_dict)

In [90]:
# current column

current_dict = {'current':'1', 'former':'2'}
mfrm.current = mfrm.current.map(current_dict)

In [91]:
# IELT_exp_yrs column

IELTS_exp_yrs_dict = {'>1':'1', '1-2':'2', '3-5':'3', '6-10':'4', '11-20':'5', '>20':'6'}
mfrm.IELTS_exp_yrs = mfrm.IELTS_exp_yrs.map(IELTS_exp_yrs_dict)

In [92]:
# rater_English column

rater_English_dict = {'L1':'1', 'L2':'2'}
mfrm.rater_English = mfrm.rater_English.map(rater_English_dict)

In [93]:
# L1_range column

L1_range_dict = {'narrow':'1', 'wide':'2'}
mfrm.L1_range = mfrm.L1_range.map(L1_range_dict)

In [94]:
# prof_range column

prof_range_dict = {'narrow':'1', 'wide':'2'}
mfrm.prof_range = mfrm.prof_range.map(prof_range_dict)

In [95]:
# Add ; before first column name so not an issue for Facets

mfrm = mfrm.rename(columns={'text_id':';text_id'})

In [96]:
mfrm.head(10)

,;text_id,rater_id,band,score,duration,text_order,gender,age,education,TESOL_cert,ELT_exp_yrs,IELTS_status,current,IELTS_exp_yrs,rater_English,L1_range,prof_range
0,1,1,1,43,3,C1_B1_B2,2,1,1,2,1,1,2,4,2,2,2
1,1,1,2,43,3,C1_B1_B2,2,1,1,2,1,1,2,4,2,2,2
2,1,1,3,43,3,C1_B1_B2,2,1,1,2,1,1,2,4,2,2,2
3,1,1,4,37,3,C1_B1_B2,2,1,1,2,1,1,2,4,2,2,2
4,1,1,5,34,3,C1_B1_B2,2,1,1,2,1,1,2,4,2,2,2
5,15,1,1,53,3,C1_B1_B2,2,1,1,2,1,1,2,4,2,2,2
6,15,1,2,63,3,C1_B1_B2,2,1,1,2,1,1,2,4,2,2,2
7,15,1,3,67,3,C1_B1_B2,2,1,1,2,1,1,2,4,2,2,2
8,15,1,4,67,3,C1_B1_B2,2,1,1,2,1,1,2,4,2,2,2
9,15,1,5,63,3,C1_B1_B2,2,1,1,2,1,1,2,4,2,2,2


In [97]:
# Write out mfrm_input.csv

mfrm.to_csv('../docs/mfrm_input.csv',index=False)

[Back to top](#Processing-quantitative-survey-data)